# MINERÍA DE TEXTOS

## Alumna: Jessica Sarahi Méndez Rincón


###  Proyecto Análisis de Sentimientos y Polaridad en Tweets
###  Tema: COVID-19

In [1]:
# Importacion de librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#Tienes que descargarte las stopwords primero via nltk.download()
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

## Contexto:
    
    El año 2020 se vio afectado por el impacto a la sociedad mundial sobre una gripe de alto nivel de contagio así como la causa principal de fallecimientos alrededor del Mundo.
    
    México no ha sido exento del raro virus.
    
    Con el presente Proyecto, el objetivo primordial es desarrollar los algoritmos que ayuden a monitorizar el efecto en los comentarios que en la Red Social de Twitter se emiten, y que ayudé a dar una idea del estado de ánimo de la gente.

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQUpnfdhCKN1C3OIbNrDdyjctlRsA9t6386WWD8JkUzt_X7JiuH&usqp=CAU" alt="Alt text that describes the graphic" title="Title text" />

## Preparar los datos


In [2]:
# Importacion del archivo que se nos proporciono agregandole cabeceras que no contiene
names = ['Id', 'tweet', 'polaridad']
dataset = pd.read_csv('train.txt', sep='\t',names=names)


In [3]:
dataset.head()

,Id,tweet,polaridad
0,818942405281980417,No mames este pinche dolor que pedo? ya mejor ...,N
1,818963123646373892,"@leomall2018 Según yo era como aviso, pero aho...",NEU
2,818998133694697472,@benshorts a juzgar por mis comportamientos au...,N
3,818822556090105857,#BuenosDias mundo Twittero ya desperté y estoy...,P
4,818993011514372098,No pude resolver el rompecabezas en Los rios d...,N


In [4]:
dataset.tweet

0      No mames este pinche dolor que pedo? ya mejor ...
1      @leomall2018 Según yo era como aviso, pero aho...
2      @benshorts a juzgar por mis comportamientos au...
3      #BuenosDias mundo Twittero ya desperté y estoy...
4      No pude resolver el rompecabezas en Los rios d...
                             ...                        
985    @ladelbosque29 acude al próximo llamado que ha...
986    @Dianybony jajajaja claro que no amor!! te amo...
987    Hoy le pedí a Dios una señal realmente obvia, ...
988    El reboot de Jumanji puede romper mi corazón x...
989    @Djrossana que tengan un lindo martes y que to...
Name: tweet, Length: 990, dtype: object

In [5]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [6]:
from sklearn import metrics
Train_X = dataset.iloc[:, 1].values
Train_Y = dataset.iloc[:, 2].values
print(Train_Y)

['N' 'NEU' 'N' 'P' 'N' 'N' 'NEU' 'N' 'N' 'P' 'N' 'P' 'P' 'NEU' 'N' 'N' 'P'
 'N' 'N' 'N' 'N' 'NEU' 'N' 'N' 'P' 'N' 'N' 'N' 'NEU' 'N' 'NEU' 'NEU' 'P'
 'N' 'N' 'P' 'P' 'NEU' 'N' 'P' 'P' 'N' 'N' 'P' 'N' 'N' 'N' 'N' 'N' 'N' 'P'
 'P' 'N' 'N' 'N' 'P' 'N' 'P' 'P' 'N' 'P' 'P' 'P' 'N' 'P' 'N' 'N' 'N' 'N'
 'P' 'P' 'P' 'N' 'NEU' 'P' 'NEU' 'N' 'NEU' 'NEU' 'P' 'N' 'N' 'P' 'NEU' 'N'
 'N' 'N' 'N' 'P' 'N' 'NEU' 'N' 'N' 'N' 'P' 'NEU' 'N' 'N' 'N' 'N' 'NEU' 'P'
 'NEU' 'P' 'N' 'P' 'NEU' 'NEU' 'N' 'N' 'P' 'N' 'N' 'P' 'N' 'N' 'N' 'N' 'N'
 'P' 'P' 'P' 'N' 'P' 'N' 'P' 'N' 'N' 'NEU' 'NEU' 'N' 'NEU' 'P' 'P' 'NEU'
 'N' 'N' 'N' 'P' 'N' 'P' 'N' 'N' 'N' 'NEU' 'NEU' 'P' 'NEU' 'P' 'N' 'N' 'P'
 'P' 'N' 'P' 'P' 'P' 'N' 'N' 'NEU' 'P' 'NEU' 'P' 'P' 'P' 'P' 'N' 'NEU'
 'NEU' 'P' 'N' 'N' 'NEU' 'N' 'P' 'N' 'P' 'N' 'P' 'NEU' 'NEU' 'P' 'N' 'N'
 'N' 'P' 'P' 'NEU' 'N' 'P' 'N' 'N' 'P' 'N' 'N' 'P' 'N' 'NEU' 'N' 'N' 'N'
 'NEU' 'N' 'P' 'N' 'N' 'N' 'N' 'P' 'N' 'NEU' 'NEU' 'NEU' 'P' 'NEU' 'NEU'
 'N' 'N' 'N' 'P' 'N' 'NEU' 'P' 'P' 'N' 'P

In [7]:
spanish_stopwords = stopwords.words('spanish')
stemmer = SnowballStemmer('spanish')
non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))
stemmer = SnowballStemmer('spanish')

In [8]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [9]:
def tokenize(text):
    text = ''.join([c for c in text if c not in non_words])
    tokens =  word_tokenize(text)
    # stem
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

In [10]:
 
#Definimos el vectorizer de nuevo y creamos un pipeline de vectorizer -> classificador
vectorizer = CountVectorizer(
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                stop_words = spanish_stopwords)

In [11]:
#LinearSVC() es el clasificador
pipeline = Pipeline([
    ('vect', vectorizer),
    ('cls', LinearSVC()),
])
#('vect', CountVectorizer()),
#('tfidf', TfidfTransformer()),
#('clf', SGDClassifier()),   

In [12]:
# Aqui definimos el espacio de parámetros a explorar
parameters = {
    'vect__max_df': (0.5, 1.9),
    'vect__min_df': (10, 20,50),
    'vect__max_features': (500, 1000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigramas or bigramas
    'cls__C': (0.2, 0.5, 0.7),
    'cls__loss': ('hinge', 'squared_hinge'),
    'cls__max_iter': (500, 1000)
}


In [13]:
#grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1 , scoring='roc_auc')
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1 )

In [14]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder() 

In [15]:
dataset['polaridadF']=Encoder.fit_transform(dataset.polaridad)

In [16]:
dataset.head()

,Id,tweet,polaridad,polaridadF
0,818942405281980417,No mames este pinche dolor que pedo? ya mejor ...,N,0
1,818963123646373892,"@leomall2018 Según yo era como aviso, pero aho...",NEU,1
2,818998133694697472,@benshorts a juzgar por mis comportamientos au...,N,0
3,818822556090105857,#BuenosDias mundo Twittero ya desperté y estoy...,P,2
4,818993011514372098,No pude resolver el rompecabezas en Los rios d...,N,0


## Buscando parametros ideales

In [17]:
grid_search.fit(dataset.tweet, dataset.polaridadF)

c:\users\jess\appdata\local\programs\python\python37\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['algun', 'com', 'contr', 'cuand', 'desd', 'dond', 'durant', 'eram', 'estab', 'estais', 'estam', 'estan', 'estand', 'estaran', 'estaras', 'esteis', 'estem', 'esten', 'estes', 'estuv', 'fuer', 'fues', 'fuim', 'fuist', 'hab', 'habr', 'habran', 'habras', 'hast', 'hem', 'hub', 'mas', 'mia', 'mias', 'mio', 'mios', 'much', 'nad', 'nosotr', 'nuestr', 'par', 'per', 'poc', 'porqu', 'qui', 'seais', 'seam', 'sent', 'ser', 'seran', 'seras', 'si', 'sient', 'sint', 'sobr', 'som', 'suy', 'tambien', 'tant', 'ten', 'tendr', 'tendran', 'tendras', 'teng', 'tien', 'tod', 'tuv', 'tuy', 'vosotr', 'vuestr'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
c:\users\jess\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Libl

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [18]:
print("El mejor parámetro es:")
print(grid_search.best_params_)

El mejor parámetro es:
{'cls__C': 0.2, 'cls__loss': 'hinge', 'cls__max_iter': 500, 'vect__max_df': 0.5, 'vect__max_features': 500, 'vect__min_df': 10, 'vect__ngram_range': (1, 1)}


In [19]:
from sklearn.model_selection import cross_val_score


In [20]:
model = LinearSVC(C=.2, loss='hinge', max_iter=1000,
multi_class='ovr',random_state=None, penalty='l2',tol=0.0001)


In [23]:
vectorizer = CountVectorizer(
 analyzer = 'word',
 tokenizer = tokenize,
 lowercase = True,
 stop_words = spanish_stopwords,
 min_df = 10,
 max_df = 1.9,
 ngram_range=(1, 1),
 max_features=1000)

 

In [24]:
corpus_data_features = vectorizer.fit_transform(dataset.tweet)
corpus_data_features_nd = corpus_data_features.toarray()
scores = cross_val_score(
 model,
 corpus_data_features_nd[0:len(dataset)],
 y=dataset.polaridad, 
 cv=5
 )
print("Probabilidad de éxito:")
print(scores.mean())

c:\users\jess\appdata\local\programs\python\python37\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['algun', 'com', 'contr', 'cuand', 'desd', 'dond', 'durant', 'eram', 'estab', 'estais', 'estam', 'estan', 'estand', 'estaran', 'estaras', 'esteis', 'estem', 'esten', 'estes', 'estuv', 'fuer', 'fues', 'fuim', 'fuist', 'hab', 'habr', 'habran', 'habras', 'hast', 'hem', 'hub', 'mas', 'mia', 'mias', 'mio', 'mios', 'much', 'nad', 'nosotr', 'nuestr', 'par', 'per', 'poc', 'porqu', 'qui', 'seais', 'seam', 'sent', 'ser', 'seran', 'seras', 'si', 'sient', 'sint', 'sobr', 'som', 'suy', 'tambien', 'tant', 'ten', 'tendr', 'tendran', 'tendras', 'teng', 'tien', 'tod', 'tuv', 'tuy', 'vosotr', 'vuestr'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Probabilidad de éxito:
0.6181818181818182


In [25]:
pipeline = Pipeline([
 ('vect', CountVectorizer(
 analyzer='word',
 tokenizer=tokenize,
 lowercase=True,
 stop_words=spanish_stopwords,
 min_df=10,
 max_df=1.9,
 ngram_range=(1, 1),
 max_features=1000
 )),
 ('cls', LinearSVC(
 C=.2,
 loss='squared_hinge',
 max_iter=1000,
 multi_class='ovr',
 random_state=None,
 penalty='l2',
 tol=0.0001
 )),
])
pipeline.fit(dataset.tweet, dataset.polaridad)


c:\users\jess\appdata\local\programs\python\python37\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['algun', 'com', 'contr', 'cuand', 'desd', 'dond', 'durant', 'eram', 'estab', 'estais', 'estam', 'estan', 'estand', 'estaran', 'estaras', 'esteis', 'estem', 'esten', 'estes', 'estuv', 'fuer', 'fues', 'fuim', 'fuist', 'hab', 'habr', 'habran', 'habras', 'hast', 'hem', 'hub', 'mas', 'mia', 'mias', 'mio', 'mios', 'much', 'nad', 'nosotr', 'nuestr', 'par', 'per', 'poc', 'porqu', 'qui', 'seais', 'seam', 'sent', 'ser', 'seran', 'seras', 'si', 'sient', 'sint', 'sobr', 'som', 'suy', 'tambien', 'tant', 'ten', 'tendr', 'tendran', 'tendras', 'teng', 'tien', 'tod', 'tuv', 'tuy', 'vosotr', 'vuestr'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.9,
                                 max_features=1000, min_df=10,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=['de', 'la', 'que', 'el', 'en', 'y',
                                             'a', 'los', 'del', 'se', 'las',
                                             'por', 'un', 'para', 'con', 'no'...
                                             'o', 'este', 'sí', 'porque', ...],
                                 strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x0000019CA00D3558>,
              

In [29]:
tweets['polarity'] = pipeline.predict(tweets.tweet)

NameError: name 'tweets' is not defined